# Declarative Data Visualisation
This strategy is "declarative" in nature. Instead of writing a "program" in the traditional sense, we write instead a JSON "spec" which is fed to an interpreter. The interpreter deals with the tedious mathematics, getting lines on screens etc. Libraries which follow this approach include, but are not limited to 

- [Vega](https://vega.github.io)
- [Vega lite](https://vega.github.io/vega-lite)
- [Apache E-charts](https://echarts.apache.org/en/index.html)
- [Plotly](https://plotly.com/graphing-libraries/)

Co-incidentally, all of these libraries, use JSON as the "spec format". There's actually nothing really special about the choice of vega here... other than it's convieniently built right into Jupyter :-).

In [3]:
private val spec = ujson.read("""{
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "description": "A simple bar chart with embedded data.",
    "data": {
      "values": [
        {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43}
      ]
    },
    "mark": "bar",
    "encoding": {
      "x": {"field": "a", "type": "nominal", "axis": {"labelAngle": 0}},
      "y": {"field": "b", "type": "quantitative"}
    },
    "width" : 300
}
""")

spec("data")("values") = ujson.Arr(
  ujson.Obj("a" -> "Fabulous", "b" -> -10),
  ujson.Obj("a" -> "Charting", "b" -> 10),
  ujson.Obj("a" -> "Potential", "b" -> 100)
)


kernel.publish.display(
  almond.interpreter.api.DisplayData(
        data = Map(
          "application/vnd.vega.v5+json" -> ujson.write(spec)
        )
  )
)

Now we changed the data! By mutating an object... if you like immutable scala, maybe that part was painful on the eyeballs :-). 

However, I would not rush to discard this as a plotting solution - looks nasty, but surprisingly, solves the general problem. By reading the vega docs, you can plot _anything_ (that vega can plot) like this. I hesitate to point out that "making the correct JSON" is a testable problem... 

Also, this strategy trivially cross compiles to scala JS. It's easy to setup if you are willing to totally plagarise [SJRDs example](https://github.com/sjrd/scalajs-sbt-vite-laminar-chartjs-example), and change it slightly to [bundle vega embed](https://github.com/Quafadas/scalajs-sbt-vite-laminar-chartjs-example) and write a [simple facade](https://github.com/Quafadas/dedav4s/blob/861c3fa38f41084f9d2e1ea168da40aab22eccf5/core/js/src/main/scala/viz/vega/facades/VegaEmbed.scala#L83). It's also pretty easy to embed in something like play! if you have static routes - just pump out the JSON into the template and include the libraries in the header of the page.

So you already have "full stack" embedabble plots, and the only library you need, is ~~ujson~~ any Json library. 

With ujson, it's easy to mess around in Jupyter / ammonite JVM land then use the _exact same code_ to publish in your full stack project. 

I claim that, if the use case is a single, complex chart, and you're willing to
- solve the meta-problems of getting vega libs in the right place
- write the facade
- use mutable stuff

Then we're done. Surpisingly, I claim this solution is significantly less sh*t than it would appear at first blush - I've used it to visualise (on the fly) the [high dimensional characsterics](https://vega.github.io/vega/examples/brushing-scatter-plots/) of a portfolio of reinsurance contracts...

For posterity, my first attempt at using vega from scala, was a [bunch of ammonite scripts](https://github.com/Quafadas/nkd-vis) that explored this strategy - but that's a historical artefact and "part of the journey" rather than something to actually go read.